### Imports

In [57]:
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog
from pathlib import Path

### Input Raw File and Decoder File

In [58]:
file = "LIV_53_QCHW2_DNS-LIVTKCOD_LCRVCOD250-DNS_raw20250130133124.csv"
decoder_file = "QC WAFER_LAYOUT 24Dec.csv"
# Define the current script's directory
current_dir = Path(os.getcwd())

# Move to the root directory
root_dir = current_dir.parents[0]  # Adjust the number based on your folder structure

# Define the path to the required subfolder
file_path = root_dir / "LIV_Raw_Files" / file
decoder_file_path = root_dir / "decoders" / decoder_file

# Read the CSV file
df = pd.read_csv(file_path, skiprows=19)

# Get the directory of the file
folder = file_path.parent
print(f"Directory: {folder}")

# df.head(5)

Directory: c:\Users\762093\OneDrive - Seagate Technology\Documents\LIV\liv-cod-algorithm\LIV_Raw_Files


### Select Required Colums

In [59]:
# Get column names
col_names = df.columns

# Find columns containing "Vf" or "PD"
selected_cols = [col for col in col_names if "Vf" in col or "PD" in col]

# Subset the data frame with selected columns
df_subset = df[selected_cols]

# Find and delete columns containing "Vf@" or "PD@"
cols_to_delete = [col for col in df_subset.columns if "Vf@" in col or "PD@" in col]
df_subset.drop(columns=cols_to_delete, inplace=True)

# df_subset.head(10)

C:\Users\762093\AppData\Local\Temp\1\ipykernel_23624\355553718.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns=cols_to_delete, inplace=True)


### Transpose

In [60]:
# Transpose the data frame
df_transposed = df_subset.transpose()

# Reset index to make the transposed columns into rows
df_transposed.reset_index(inplace=True)

# Set the first row as column names and remove the first row
df_transposed.columns = df_transposed.iloc[0]
df_transposed = df_transposed[1:]

# Rename the first column to 'Label' and the rest to integers starting from 1
new_columns = ["Label"] + list(range(1, len(df_transposed.columns)))
df_transposed.columns = new_columns

# Display the first 10 rows of df_transposed
df_transposed.head(10)

,Label,1,2,3,4,5,6,7,8,9,...,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518
1,Vf2,1.398021,1.395625,1.398368,1.399914,1.396630,1.400571,1.395393,1.398253,1.394620,...,1.396437,1.392457,1.393732,1.393384,0.834385,5.062486,5.061738,5.062486,5.061738,5.062486
2,Vf3,1.446096,1.444628,1.445826,1.446753,1.446212,1.447255,1.446483,1.445710,1.445748,...,1.440068,1.441498,1.442387,1.441962,0.879949,5.061296,5.060568,5.061296,5.060568,5.061296
3,Vf4,1.475857,1.474195,1.474659,1.475161,1.473345,1.475895,1.474775,1.476629,1.473577,...,1.467781,1.467433,1.468129,1.470292,0.919523,5.060106,5.059398,5.060106,5.059398,5.060106
4,Vf5,1.494875,1.494721,1.496459,1.495725,1.495609,1.496962,1.495648,1.497348,1.496073,...,1.489775,1.488190,1.489079,1.491436,0.955698,5.058916,5.058228,5.058916,5.058228,5.058916
5,Vf6,1.512425,1.513430,1.514589,1.514164,1.514396,1.513855,1.514010,1.514357,1.514048,...,1.506900,1.506011,1.506320,1.508214,0.981556,5.057726,5.057058,5.057726,5.057058,5.057726
6,Vf7,1.528623,1.528546,1.529821,1.528082,1.530207,1.530632,1.529010,1.530014,1.529396,...,1.521359,1.520586,1.521513,1.525416,0.999608,5.056536,5.055888,5.056536,5.055888,5.056536
7,Vf8,1.543507,1.543700,1.542889,1.542889,1.543855,1.543816,1.544898,1.544744,1.544048,...,1.534891,1.535316,1.535354,1.539025,1.031378,5.055346,5.054718,5.055346,5.054718,5.055346
8,Vf9,1.555996,1.556691,1.555725,1.557619,1.556691,1.556768,1.556962,1.557155,1.556343,...,1.547920,1.547147,1.547109,1.550702,1.058047,5.054156,5.053548,5.054156,5.053548,5.054156
9,Vf10,1.567518,1.568484,1.568175,1.568716,1.569180,1.568716,1.568484,1.568948,1.569489,...,1.559095,1.558283,1.557704,1.562070,1.084677,5.052966,5.052378,5.052966,5.052378,5.052966
10,Vf11,1.579195,1.579929,1.580316,1.579581,1.579775,1.580702,1.580547,1.580432,1.580625,...,1.569999,1.568917,1.568260,1.572201,1.104701,5.051776,5.051208,5.051776,5.051208,5.051776


### Flatten Data

In [70]:
# Split transposed table into Vf and PD data tables
df_vf = df_transposed[df_transposed["Label"].str.contains("Vf")]
df_pd = df_transposed[df_transposed["Label"].str.contains("PD")]
# df_vf.head(10)
# df_pd.head(10)

# Drop the 'Label' column
df_vf.drop(columns=["Label"], inplace=True)
df_pd.drop(columns=["Label"], inplace=True)

# Concatenate all Voltage columns into one
n_meas = df_vf.shape[0]
n_devices = df_vf.shape[1]
df_concat_vf = pd.DataFrame(df_vf.values.flatten(), columns=["Vf"])

# Create TOUCHDOWN column
df_concat_vf["TOUCHDOWN"] = [i // n_meas + 1 for i in range(n_meas * n_devices)]

# Concatenate all PD columns into one
df_concat_pd = pd.DataFrame(df_pd.values.flatten(), columns=["PD"])

# Cartesian join of Vf and PD data tables
df_raw_sweeps = pd.concat([df_concat_vf, df_concat_pd], axis=1)
# df_raw_sweeps.head(270)

C:\Users\762093\AppData\Local\Temp\1\ipykernel_23624\1286412432.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vf.drop(columns=["Label"], inplace=True)
C:\Users\762093\AppData\Local\Temp\1\ipykernel_23624\1286412432.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pd.drop(columns=["Label"], inplace=True)


### Add Device Coords

In [71]:
# Add device coordinates from original RAW file
if "TOUCHDOWN" in df.columns and "STX_WAFER_X_UM" in df.columns and "STX_WAFER_Y_UM" in df.columns:
    df_raw_sweeps = df_raw_sweeps.merge(
        df[["TOUCHDOWN", "STX_WAFER_X_UM", "STX_WAFER_Y_UM"]], on="TOUCHDOWN", how="left"
    )
else:
    print("Required columns for merging device coordinates are missing in the original RAW file.")

if decoder_file_path.exists():
    df_decoder = pd.read_csv(decoder_file_path)
    # Update with decoder to get TE_LABEL etc.
    if "YMIN" in df_decoder.columns and "XMIN" in df_decoder.columns:
        df_raw_sweeps = df_raw_sweeps.merge(
            df_decoder[["YMIN", "XMIN", "TE_LABEL", "TYPE"]],
            left_on=["STX_WAFER_Y_UM", "STX_WAFER_X_UM"],
            right_on=["YMIN", "XMIN"],
            how="left",
        ).drop(columns=["YMIN", "XMIN"])
    else:
        print("Required columns for merging decoder data are missing in the decoder file.")
else:
    print(f"Decoder file not found at {decoder_file_path}")

# Rename the columns
df_raw_sweeps.rename(columns={"STX_WAFER_X_UM": "X_UM", "STX_WAFER_Y_UM": "Y_UM"}, inplace=True)

# Add current column as a repeating sequence of length n_meas
df_raw_sweeps["LDI_mA"] = [i % n_meas + 1 for i in range(len(df_raw_sweeps))]

# Display the first 10 rows of df_raw_sweeps
df_raw_sweeps.head(20)

,Vf,TOUCHDOWN,PD,X_UM,Y_UM,TE_LABEL,TYPE,LDI_mA
0,1.398021,1.0,0.171087,-32339.0,-48416.0,64N6V,BL LASER,1
1,1.395625,1.0,0.174109,-32339.0,-48416.0,64N6V,BL LASER,2
2,1.398368,1.0,0.165045,-32339.0,-48416.0,64N6V,BL LASER,3
3,1.399914,1.0,0.166555,-32339.0,-48416.0,64N6V,BL LASER,4
4,1.396630,1.0,0.165045,-32339.0,-48416.0,64N6V,BL LASER,5
5,1.400571,1.0,0.162023,-32339.0,-48416.0,64N6V,BL LASER,6
6,1.395393,1.0,0.159001,-32339.0,-48416.0,64N6V,BL LASER,7
7,1.398253,1.0,0.163534,-32339.0,-48416.0,64N6V,BL LASER,8
8,1.394620,1.0,0.162023,-32339.0,-48416.0,64N6V,BL LASER,9
9,1.399180,1.0,0.163534,-32339.0,-48416.0,64N6V,BL LASER,10
